In [1]:
import pyterrier as pt
if not pt.started():
    pt.init()

PyTerrier 0.9.2 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [2]:
from pyterrier_colbert.ranking import ColBERTFactory
pytcolbert = ColBERTFactory("http://www.dcs.gla.ac.uk/~craigm/ecir2021-tutorial/colbert_model_checkpoint.zip", "./msmarco_index", "msmarco", gpu=True)

/home/stu1/s8/gfp4857/miniconda3/envs/project/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at bert-base-uncased were not used when initializing ColBERT: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing ColBERT from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ColBERT from the checkpoint of a model that you expect to

[May 04, 20:09:01] #> Loading model checkpoint.
[May 04, 20:09:01] #> Loading checkpoint http://www.dcs.gla.ac.uk/~craigm/ecir2021-tutorial/colbert_model_checkpoint.zip


/home/stu1/s8/gfp4857/miniconda3/envs/project/lib/python3.11/site-packages/torch/hub.py:665: UserWarning: Falling back to the old format < 1.6. This support will be deprecated in favor of default zipfile format introduced in 1.6. Please redo torch.save() to save it in the new zipfile format.
  warnings.warn('Falling back to the old format < 1.6. This support will be '


[May 04, 20:09:29] #> checkpoint['epoch'] = 0
[May 04, 20:09:29] #> checkpoint['batch'] = 44500


In [3]:
import numpy as np
from ir_measures import RR

msmarco_ds = pt.get_dataset("msmarco_passage")

# Get list 
qids = list(msmarco_ds.get_qrels("dev").loc[msmarco_ds.get_qrels("dev")['docno'].astype(int) < 100000]['qid'])

results = []

for p in np.arange(0.0, 0.51, 0.05):
    token_ids = pytcolbert.top_p_tokens(p)
    dense_e2e = pytcolbert.end_to_end(token_ids)


    print(f"{p=}")
    results += [pt.Experiment(
        [dense_e2e],
        msmarco_ds.get_topics("dev").loc[msmarco_ds.get_topics("dev")['qid'].str.contains('|'.join(qids), na=False)].head(10),
        msmarco_ds.get_qrels("dev"),
        eval_metrics=["map", RR@10],
        names=[f"ColBERT ({p=})"]
    )]

    print(results[-1])

    del dense_e2e



[May 04, 20:09:30] #> Loading the FAISS index from ./msmarco_index/msmarco/ivfpq.faiss ..
[May 04, 20:09:30] #> Building the emb2pid mapping..
[May 04, 20:09:30] len(self.emb2pid) = 7409337
DEBUG>>> token_ids_to_prune={1012}
Loading reranking index, memtype=mem


Loading index shards to memory: 100%|██████████| 1/1 [00:00<00:00,  1.86shard/s]


DEBUG>>> vectors_ids_to_prune=array([     23,      54,      73, ..., 7409299, 7409331, 7409335])
DEBUG>>> pre-remove: 7409337
DEBUG>>> post-remove: 7024102
      name       map     RR@10
0  ColBERT  0.378704  0.462037
DEBUG>>> token_ids_to_prune={1012}
DEBUG>>> vectors_ids_to_prune=array([     23,      54,      73, ..., 7409299, 7409331, 7409335])
DEBUG>>> pre-remove: 7409337
DEBUG>>> post-remove: 7024102
      name       map     RR@10
0  ColBERT  0.378704  0.462037
DEBUG>>> token_ids_to_prune={1010, 1996, 1012}
DEBUG>>> vectors_ids_to_prune=array([      2,      13,      16, ..., 7409326, 7409331, 7409335])
DEBUG>>> pre-remove: 7409337
DEBUG>>> post-remove: 6425895
      name       map     RR@10
0  ColBERT  0.378704  0.462037
DEBUG>>> token_ids_to_prune={1010, 1996, 1012, 1997}
DEBUG>>> vectors_ids_to_prune=array([      2,       4,      13, ..., 7409328, 7409331, 7409335])
DEBUG>>> pre-remove: 7409337
DEBUG>>> post-remove: 6262190
      name       map     RR@10
0  ColBERT  0.378704  0.

In [ ]:
token_ids = pytcolbert.top_p_tokens(0.3)
dense_e2e = pytcolbert.end_to_end(token_ids)

In [ ]:
msmarco_ds = pt.get_dataset("msmarco_passage")

# Get list 
qids = list(msmarco_ds.get_qrels("dev").loc[msmarco_ds.get_qrels("dev")['docno'].astype(int) < 100000]['qid'])

pt.Experiment(
    [dense_e2e],
    msmarco_ds.get_topics("dev").loc[msmarco_ds.get_topics("dev")['qid'].str.contains('|'.join(qids), na=False)].head(10),
    msmarco_ds.get_qrels("dev"),
    eval_metrics=["map. mrr@10"],
    names=["ColBERT"]
)


In [ ]:
result = dense_e2e.search("cost of endless pools swim spa")

In [ ]:
dense_e2e.search("what is pcnt")